In [1]:
# Load required packages
library(dplyr,warn.conflicts=F)
library(stringr,warn.conflicts=F)
library(psych,warn.conflicts=F)
library(lplyr,warn.conflicts=F)
library(tidyr,warn.conflicts=F)

In [2]:
library(tictoc,warn.conflicts=F)

#### From Matchups stats to Teams stats

In [3]:
# load the stats and keep 2014+
data.regulars <- readRDS('datafiles_rds/RegularSeasonDetailedResults.rds')
data.regulars %>% filter(Season>2013) -> data.regulars

In [4]:
data.regulars %>% dim %>% cat('\n')
data.regulars %>% colnames %>% cat('\n')

26885 34 
Season DayNum WTeamID WScore LTeamID LScore WLoc NumOT WFGM WFGA WFGM3 WFGA3 WFTM WFTA WOR WDR WAst WTO WStl WBlk WPF LFGM LFGA LFGM3 LFGA3 LFTM LFTA LOR LDR LAst LTO LStl LBlk LPF 


In [5]:
data.regulars %>% group_by(Season) %>% summarise(n=n())

Season,n
2014,5362
2015,5354
2016,5369
2017,5395
2018,5405


In [6]:
data.regulars %>% head

Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,⋯,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
2014,4,1102,79,1119,68,N,0,26,49,⋯,44,10,13,14,20,16,13,4,5,24
2014,4,1103,72,1157,63,H,0,25,59,⋯,14,11,18,15,22,6,20,8,4,22
2014,4,1107,74,1373,62,A,0,21,52,⋯,24,12,23,20,24,12,16,2,6,26
2014,4,1112,73,1142,62,H,0,24,42,⋯,26,11,13,10,16,13,11,4,0,26
2014,4,1113,96,1420,61,H,0,27,57,⋯,23,11,25,11,24,12,16,8,1,30
2014,4,1116,99,1188,65,H,0,33,64,⋯,25,6,9,15,19,10,23,6,1,24


In [7]:
data.regulars %>%
    select(-WLoc,-NumOT) -> tmp01
tmp01 %>%
    select(-LTeamID) %>%
    mutate(OW=1,OL=0,DW=0,DL=1) %>%
    rename(TeamID=WTeamID) %>%
    rename_at(vars(starts_with('W')),list(~str_replace(.,'^W','O'))) %>% 
    rename_at(vars(starts_with('L')),list(~str_replace(.,'^L','D'))) -> group1
tmp01 %>%
    select(-WTeamID) %>%
    mutate(OW=0,OL=1,DW=1,DL=0) %>%
    rename(TeamID=LTeamID) %>%
    rename_at(vars(starts_with('L')),list(~str_replace(.,'^L','O'))) %>%
    rename_at(vars(starts_with('W')),list(~str_replace(.,'^W','D'))) -> group2
group1 %>% 
    bind_rows(group2) %>% 
    arrange(Season,DayNum,TeamID) -> teams.stats

In [8]:
teams.stats %>% filter(TeamID==1102) %>% head

Season,DayNum,TeamID,OScore,DScore,OFGM,OFGA,OFGM3,OFGA3,OFTM,⋯,DDR,DAst,DTO,DStl,DBlk,DPF,OW,OL,DW,DL
2014,4,1102,79,68,26,49,10,15,17,⋯,20,16,13,4,5,24,1,0,0,1
2014,5,1102,63,71,23,53,5,22,12,⋯,27,11,12,10,4,20,0,1,1,0
2014,10,1102,82,84,27,57,12,26,16,⋯,27,15,12,5,8,25,0,1,1,0
2014,13,1102,67,64,19,45,9,21,20,⋯,20,13,15,12,6,23,1,0,0,1
2014,23,1102,60,83,20,43,4,13,16,⋯,21,24,8,10,6,22,0,1,1,0
2014,26,1102,57,81,19,46,6,22,13,⋯,28,20,14,5,3,21,0,1,1,0


In [9]:
teams.stats %>% group_by(Season) %>% summarise(n=n())

Season,n
2014,10724
2015,10708
2016,10738
2017,10790
2018,10810


#### Compute regular new features

In [10]:
# compute cumulative and average stats
teams.stats %>%
    group_by(Season,TeamID) %>%
    mutate_at(vars(starts_with('O'),starts_with('D')),list(~cumsum,~cummean)) %>%
    select(-c(OW,OL,DW,DL,OScore,DScore,OFGM,OFGA,OFGM3,OFGA3,OFTM,OFTA,OOR,ODR,OAst,OTO,OStl,OBlk,OPF,DFGM,DFGA,DFGM3,DFGA3,DFTM,DFTA,DOR,DDR,DAst,DTO,DStl,DBlk,DPF)) %>%
    rename_all(list(~str_replace(.,'_cumsum','Sum'))) %>%
    rename_all(list(~str_replace(.,'_cummean','Avg'))) %>%
    ungroup() -> teams.stats.cumul

In [11]:
teams.stats.cumul %>% dim %>% cat('\n')
teams.stats.cumul %>% colnames %>% cat('\n')

53770 69 
Season DayNum TeamID OScoreSum OFGMSum OFGASum OFGM3Sum OFGA3Sum OFTMSum OFTASum OORSum ODRSum OAstSum OTOSum OStlSum OBlkSum OPFSum OWSum OLSum DayNumSum DScoreSum DFGMSum DFGASum DFGM3Sum DFGA3Sum DFTMSum DFTASum DORSum DDRSum DAstSum DTOSum DStlSum DBlkSum DPFSum DWSum DLSum OScoreAvg OFGMAvg OFGAAvg OFGM3Avg OFGA3Avg OFTMAvg OFTAAvg OORAvg ODRAvg OAstAvg OTOAvg OStlAvg OBlkAvg OPFAvg OWAvg OLAvg DayNumAvg DScoreAvg DFGMAvg DFGAAvg DFGM3Avg DFGA3Avg DFTMAvg DFTAAvg DORAvg DDRAvg DAstAvg DTOAvg DStlAvg DBlkAvg DPFAvg DWAvg DLAvg 


In [12]:
teams.stats.cumul %>% filter(Season==2014,TeamID==1102) %>% head
teams.stats.cumul %>% filter(Season==2015,TeamID==1102) %>% head

Season,DayNum,TeamID,OScoreSum,OFGMSum,OFGASum,OFGM3Sum,OFGA3Sum,OFTMSum,OFTASum,⋯,DFTAAvg,DORAvg,DDRAvg,DAstAvg,DTOAvg,DStlAvg,DBlkAvg,DPFAvg,DWAvg,DLAvg
2014,4,1102,79,26,49,10,15,17,26,⋯,13.00000,14.00000,20.00000,16.00,13.00000,4.000000,5.000000,24.0,0.0000000,1.0000000
2014,5,1102,142,49,102,15,37,29,45,⋯,19.50000,13.00000,23.50000,13.50,12.50000,7.000000,4.500000,22.0,0.5000000,0.5000000
2014,10,1102,224,76,159,27,63,45,71,⋯,26.66667,11.33333,24.66667,14.00,12.33333,6.333333,5.666667,23.0,0.6666667,0.3333333
2014,13,1102,291,95,204,36,84,65,97,⋯,23.25000,11.25000,23.50000,13.75,13.00000,7.750000,5.750000,23.0,0.5000000,0.5000000
2014,23,1102,351,115,247,40,97,81,123,⋯,23.80000,11.20000,23.00000,15.80,12.00000,8.200000,5.800000,22.8,0.6000000,0.4000000
2014,26,1102,408,134,293,46,119,94,142,⋯,23.83333,11.66667,23.83333,16.50,12.33333,7.666667,5.333333,22.5,0.6666667,0.3333333


Season,DayNum,TeamID,OScoreSum,OFGMSum,OFGASum,OFGM3Sum,OFGA3Sum,OFTMSum,OFTASum,⋯,DFTAAvg,DORAvg,DDRAvg,DAstAvg,DTOAvg,DStlAvg,DBlkAvg,DPFAvg,DWAvg,DLAvg
2015,11,1102,78,31,62,6,19,10,15,⋯,17.00000,17.00000,20.00000,18.00000,15.00000,6.000000,4.000000,21.00,1.0000000,0.0000000
2015,12,1102,146,55,105,15,39,21,29,⋯,18.00000,15.00000,14.50000,12.50000,13.00000,6.000000,2.000000,19.00,0.5000000,0.5000000
2015,22,1102,199,73,160,19,59,34,49,⋯,20.00000,13.66667,19.66667,12.66667,13.66667,6.333333,3.666667,20.00,0.6666667,0.3333333
2015,27,1102,261,96,207,27,77,42,58,⋯,20.50000,14.50000,19.00000,11.50000,14.75000,7.000000,3.750000,18.25,0.7500000,0.2500000
2015,30,1102,320,116,260,32,101,56,83,⋯,18.60000,14.00000,18.40000,11.00000,14.20000,6.600000,3.200000,18.80,0.6000000,0.4000000
2015,35,1102,397,143,310,41,126,70,100,⋯,17.33333,14.16667,18.16667,11.50000,14.33333,6.500000,3.333333,19.00,0.5000000,0.5000000


In [13]:
# create missing row
teams.stats.cumul %>% complete(Season=2014:2018,DayNum=1:140,TeamID) %>% 
    group_by(Season,TeamID) %>%
    fill(everything()) %>%
    ungroup() -> teams.stats.cumul

In [14]:
# store
teams.stats.cumul %>% dim %>% cat('\n')
teams.stats.cumul %>% colnames %>% cat('\n')
teams.stats.cumul %>% saveRDS('datafiles_rds/TeamOffDefStats.rds')

245700 69 
Season DayNum TeamID OScoreSum OFGMSum OFGASum OFGM3Sum OFGA3Sum OFTMSum OFTASum OORSum ODRSum OAstSum OTOSum OStlSum OBlkSum OPFSum OWSum OLSum DayNumSum DScoreSum DFGMSum DFGASum DFGM3Sum DFGA3Sum DFTMSum DFTASum DORSum DDRSum DAstSum DTOSum DStlSum DBlkSum DPFSum DWSum DLSum OScoreAvg OFGMAvg OFGAAvg OFGM3Avg OFGA3Avg OFTMAvg OFTAAvg OORAvg ODRAvg OAstAvg OTOAvg OStlAvg OBlkAvg OPFAvg OWAvg OLAvg DayNumAvg DScoreAvg DFGMAvg DFGAAvg DFGM3Avg DFGA3Avg DFTMAvg DFTAAvg DORAvg DDRAvg DAstAvg DTOAvg DStlAvg DBlkAvg DPFAvg DWAvg DLAvg 


#### Regular matchups

In [15]:
# regular season matchups
data.regulars %>% 
    mutate(TeamID.x=ifelse(WTeamID<LTeamID,WTeamID,LTeamID),
           TeamID.y=ifelse(WTeamID>LTeamID,WTeamID,LTeamID)) %>%
    mutate(Target=ifelse(WTeamID<LTeamID,1,0)) %>%
    mutate(DayNum=DayNum-1) %>%
    select(Season,DayNum,Target,TeamID.x,TeamID.y) -> train.matchups
train.matchups %>% sample_n(5)

Season,DayNum,Target,TeamID.x,TeamID.y
2014,60,1,1153,1272
2017,10,0,1163,1447
2016,49,1,1112,1253
2014,93,0,1254,1352
2016,22,0,1400,1401


In [16]:
train.matchups %>% group_by(Season) %>% summarise(n=n())

Season,n
2014,5362
2015,5354
2016,5369
2017,5395
2018,5405


In [17]:
# regular matchups features
train.matchups %>%
    left_join(teams.stats.cumul,by=c('Season','DayNum','TeamID.x'='TeamID')) %>% 
    rename_all(list(~str_replace(.,'Sum$','Sum.x'))) %>%
    rename_all(list(~str_replace(.,'Avg$','Avg.x'))) %>%
    left_join(teams.stats.cumul,by=c('Season','DayNum','TeamID.y'='TeamID')) %>% 
    rename_all(list(~str_replace(.,'Sum$','Sum.y'))) %>%
    rename_all(list(~str_replace(.,'Avg$','Avg.y'))) %>%
    na.omit() -> train.matchups.features
train.matchups.features %>% sample_n(5)

Season,DayNum,Target,TeamID.x,TeamID.y,OScoreSum.x,OFGMSum.x,OFGASum.x,OFGM3Sum.x,OFGA3Sum.x,⋯,DFTAAvg.y,DORAvg.y,DDRAvg.y,DAstAvg.y,DTOAvg.y,DStlAvg.y,DBlkAvg.y,DPFAvg.y,DWAvg.y,DLAvg.y
2016,81,1,1226,1451,1059,368,896,107,326,⋯,16.93333,9.60000,23.80000,11.86667,11.20000,5.533333,1.533333,22.06667,0.3333333,0.66666667
2016,72,1,1225,1319,901,291,735,90,262,⋯,20.75000,12.00000,29.91667,16.33333,13.41667,7.083333,4.916667,17.83333,0.9166667,0.08333333
2016,64,0,1102,1461,839,288,671,62,202,⋯,21.30769,12.46154,25.84615,11.84615,10.84615,7.153846,2.384615,23.38462,0.6153846,0.38461538
2016,49,0,1101,1179,549,183,442,50,153,⋯,17.40000,10.80000,23.80000,10.20000,10.90000,6.600000,5.600000,19.20000,0.7000000,0.30000000
2017,88,1,1294,1381,1210,426,1011,152,446,⋯,30.00000,10.68421,28.26316,15.42105,12.68421,5.473684,4.947368,21.42105,0.8947368,0.10526316


In [18]:
train.matchups %>% group_by(Season) %>% summarise(n=n())

Season,n
2014,5362
2015,5354
2016,5369
2017,5395
2018,5405


In [19]:
dim(train.matchups.features) %>% cat('\n')
train.matchups.features %>% colnames %>% cat('\n')

25844 137 
Season DayNum Target TeamID.x TeamID.y OScoreSum.x OFGMSum.x OFGASum.x OFGM3Sum.x OFGA3Sum.x OFTMSum.x OFTASum.x OORSum.x ODRSum.x OAstSum.x OTOSum.x OStlSum.x OBlkSum.x OPFSum.x OWSum.x OLSum.x DayNumSum.x DScoreSum.x DFGMSum.x DFGASum.x DFGM3Sum.x DFGA3Sum.x DFTMSum.x DFTASum.x DORSum.x DDRSum.x DAstSum.x DTOSum.x DStlSum.x DBlkSum.x DPFSum.x DWSum.x DLSum.x OScoreAvg.x OFGMAvg.x OFGAAvg.x OFGM3Avg.x OFGA3Avg.x OFTMAvg.x OFTAAvg.x OORAvg.x ODRAvg.x OAstAvg.x OTOAvg.x OStlAvg.x OBlkAvg.x OPFAvg.x OWAvg.x OLAvg.x DayNumAvg.x DScoreAvg.x DFGMAvg.x DFGAAvg.x DFGM3Avg.x DFGA3Avg.x DFTMAvg.x DFTAAvg.x DORAvg.x DDRAvg.x DAstAvg.x DTOAvg.x DStlAvg.x DBlkAvg.x DPFAvg.x DWAvg.x DLAvg.x OScoreSum.y OFGMSum.y OFGASum.y OFGM3Sum.y OFGA3Sum.y OFTMSum.y OFTASum.y OORSum.y ODRSum.y OAstSum.y OTOSum.y OStlSum.y OBlkSum.y OPFSum.y OWSum.y OLSum.y DayNumSum.y DScoreSum.y DFGMSum.y DFGASum.y DFGM3Sum.y DFGA3Sum.y DFTMSum.y DFTASum.y DORSum.y DDRSum.y DAstSum.y DTOSum.y DStlSum.y DBlkSum.y DPF

In [118]:
train.matchups.features %>%
    select(-Season,-DayNum,-TeamID.x,-TeamID.y) %>%
    write.csv('kerasfiles/train.matchups.ODstats.csv',row.names=F)

#### Delta Ratings

In [34]:
train.matchups %>% 
    inner_join(teams.stats.cumul,by=c('Season','DayNum','TeamID.x'='TeamID')) %>%
    mutate(pos=1) -> group1
train.matchups %>% 
    inner_join(teams.stats.cumul,by=c('Season','DayNum','TeamID.y'='TeamID')) %>%
    mutate(pos=2) -> group2
group1 %>% 
    bind_rows(group2) %>% 
    arrange(pos) %>% 
    group_by(Season,DayNum,TeamID.x,TeamID.y) %>% 
    mutate_at(vars(OScoreSum:DLAvg),list(~lag(.)-.)) %>% 
    ungroup %>% 
    filter(pos==2) %>%
    na.omit() %>%
    select(-pos) -> df.ratingdelta

In [35]:
df.ratingdelta %>% head

Season,DayNum,Target,TeamID.x,TeamID.y,OScoreSum,OFGMSum,OFGASum,OFGM3Sum,OFGA3Sum,⋯,DFTAAvg,DORAvg,DDRAvg,DAstAvg,DTOAvg,DStlAvg,DBlkAvg,DPFAvg,DWAvg,DLAvg
2014,4,1,1119,1154,-3,-1,-4,6,23,⋯,6,-13,3,2,7,-7,-5,-2,0,0
2014,4,1,1184,1244,26,8,1,5,8,⋯,14,6,-1,-1,1,-5,-1,1,-1,1
2014,4,1,1218,1444,15,4,4,-5,-6,⋯,-23,-16,-3,-1,5,-1,-2,5,0,0
2014,4,1,1258,1355,8,4,6,-1,-5,⋯,-2,0,-3,-1,0,-6,1,-1,-1,1
2014,4,1,1308,1398,9,-2,-2,0,-4,⋯,-17,-4,-13,-6,1,-7,-2,9,0,0
2014,4,0,1213,1360,1,-1,-1,-4,-3,⋯,0,5,-5,-3,-1,-2,-2,0,1,-1


In [39]:
group1 %>% filter(Season==2016,DayNum==116,TeamID.x==1255)
group2 %>% filter(Season==2016,DayNum==116,TeamID.y==1347)
df.ratingdelta %>% filter(Season==2016,DayNum==116,TeamID.x==1255)

Season,DayNum,Target,TeamID.x,TeamID.y,OScoreSum,OFGMSum,OFGASum,OFGM3Sum,OFGA3Sum,⋯,DORAvg,DDRAvg,DAstAvg,DTOAvg,DStlAvg,DBlkAvg,DPFAvg,DWAvg,DLAvg,pos
2016,116,1,1255,1347,1904,679,1533,188,529,⋯,10.03704,25.07407,15.37037,12.25926,7.333333,2.888889,19.11111,0.8148148,0.1851852,1


Season,DayNum,Target,TeamID.x,TeamID.y,OScoreSum,OFGMSum,OFGASum,OFGM3Sum,OFGA3Sum,⋯,DORAvg,DDRAvg,DAstAvg,DTOAvg,DStlAvg,DBlkAvg,DPFAvg,DWAvg,DLAvg,pos
2016,116,1,1255,1347,1930,716,1615,184,544,⋯,10.2963,24.2963,12.03704,11.81481,7.185185,3.37037,17.2963,0.4814815,0.5185185,2


Season,DayNum,Target,TeamID.x,TeamID.y,OScoreSum,OFGMSum,OFGASum,OFGM3Sum,OFGA3Sum,⋯,DFTAAvg,DORAvg,DDRAvg,DAstAvg,DTOAvg,DStlAvg,DBlkAvg,DPFAvg,DWAvg,DLAvg
2016,116,1,1255,1347,-26,-37,-82,4,-15,⋯,1.962963,-0.2592593,0.7777778,3.333333,0.4444444,0.1481481,-0.4814815,1.814815,0.3333333,-0.3333333


In [40]:
df.ratingdelta %>%
    select(-Season,-DayNum,-TeamID.x,-TeamID.y) %>%
    write.csv('kerasfiles/train.matchups.ODDeltaStats.csv',row.names=F)

#### Tourney matchups

In [42]:
# load the stats and keep 2014+
data.tourney <- readRDS('datafiles_rds/NCAATourneyDetailedResults.rds')
data.tourney %>% filter(Season>2013) -> data.tourney

In [43]:
data.tourney %>% group_by(Season) %>% summarise(n=n())

Season,n
2014,67
2015,67
2016,67
2017,67
2018,67


In [44]:
data.test.matchups <- readRDS('datafiles_rds/TourneyMatchups.rds')

In [45]:
# tourney season matchups
data.test.matchups %>% 
    mutate(ID=str_c(Season,'_',TeamID.x,'_',TeamID.y)) %>%
    mutate(DayNum=140) %>%
    select(ID,Season,DayNum,TeamID.x,TeamID.y) -> test.matchups
test.matchups %>% sample_n(5)

ID,Season,DayNum,TeamID.x,TeamID.y
2018_1120_1281,2018,140,1120,1281
2017_1245_1423,2017,140,1245,1423
2017_1308_1407,2017,140,1308,1407
2014_1291_1344,2014,140,1291,1344
2017_1343_1437,2017,140,1343,1437


In [46]:
# tourney season truth
data.tourney %>% 
    mutate(TeamID.x=ifelse(WTeamID<LTeamID,WTeamID,LTeamID),
           TeamID.y=ifelse(WTeamID>LTeamID,WTeamID,LTeamID)) %>%
    mutate(Target=ifelse(WTeamID<LTeamID,1,0)) %>%
    mutate(ID=str_c(Season,'_',TeamID.x,'_',TeamID.y)) %>%
    mutate(Tourney=1) %>%
    select(ID,Tourney,Target) -> test.truth
test.truth %>% sample_n(5)

ID,Tourney,Target
2018_1314_1401,1,0
2014_1277_1438,1,1
2014_1235_1314,1,1
2018_1196_1382,1,1
2017_1124_1425,1,1


In [47]:
test.matchups %>%
    left_join(test.truth,by='ID') %>% 
    mutate_which(is.na(Target),Target=0,Tourney=0) -> test.matchups

In [48]:
test.matchups %>% group_by(Season) %>% summarise(n=n())

Season,n
2014,2278
2015,2278
2016,2278
2017,2278
2018,2278


In [49]:
# tourney matchups features
test.matchups %>%
    left_join(teams.stats.cumul,by=c('Season','DayNum','TeamID.x'='TeamID')) %>% 
    rename_all(list(~str_replace(.,'Sum$','Sum.x'))) %>%
    rename_all(list(~str_replace(.,'Avg$','Avg.x'))) %>%
    left_join(teams.stats.cumul,by=c('Season','DayNum','TeamID.y'='TeamID')) %>% 
    rename_all(list(~str_replace(.,'Sum$','Sum.y'))) %>%
    rename_all(list(~str_replace(.,'Avg$','Avg.y'))) %>%
    na.omit() -> test.matchups.features
test.matchups.features %>% sample_n(5)

ID,Season,DayNum,TeamID.x,TeamID.y,Tourney,Target,OScoreSum.x,OFGMSum.x,OFGASum.x,⋯,DFTAAvg.y,DORAvg.y,DDRAvg.y,DAstAvg.y,DTOAvg.y,DStlAvg.y,DBlkAvg.y,DPFAvg.y,DWAvg.y,DLAvg.y
2016_1201_1403,2016,140,1201,1403,0,0,2388,836,1942,⋯,18.90323,11.483871,22.22581,11.22581,12.77419,6.354839,3.290323,21.09677,0.3870968,0.6129032
2018_1181_1452,2018,140,1181,1452,0,0,2795,1008,2044,⋯,23.38235,10.235294,24.23529,12.23529,16.50000,5.470588,3.941176,18.76471,0.2941176,0.7058824
2015_1246_1372,2015,140,1246,1372,0,0,2547,883,1884,⋯,24.33333,9.533333,19.83333,10.43333,16.56667,5.766667,3.433333,21.50000,0.1333333,0.8666667
2014_1264_1344,2014,140,1264,1344,0,0,2476,805,1786,⋯,19.64706,10.382353,22.61765,11.97059,11.00000,5.676471,3.176471,19.70588,0.3235294,0.6764706
2015_1412_1452,2015,140,1412,1452,0,0,2247,782,1827,⋯,26.03125,8.906250,24.31250,11.68750,19.62500,5.781250,4.031250,22.46875,0.2812500,0.7187500


In [50]:
test.matchups %>% group_by(Season) %>% summarise(n=n())

Season,n
2014,2278
2015,2278
2016,2278
2017,2278
2018,2278


In [51]:
dim(test.matchups.features) %>% cat('\n')
test.matchups.features %>% colnames %>% cat('\n')

11390 139 
ID Season DayNum TeamID.x TeamID.y Tourney Target OScoreSum.x OFGMSum.x OFGASum.x OFGM3Sum.x OFGA3Sum.x OFTMSum.x OFTASum.x OORSum.x ODRSum.x OAstSum.x OTOSum.x OStlSum.x OBlkSum.x OPFSum.x OWSum.x OLSum.x DayNumSum.x DScoreSum.x DFGMSum.x DFGASum.x DFGM3Sum.x DFGA3Sum.x DFTMSum.x DFTASum.x DORSum.x DDRSum.x DAstSum.x DTOSum.x DStlSum.x DBlkSum.x DPFSum.x DWSum.x DLSum.x OScoreAvg.x OFGMAvg.x OFGAAvg.x OFGM3Avg.x OFGA3Avg.x OFTMAvg.x OFTAAvg.x OORAvg.x ODRAvg.x OAstAvg.x OTOAvg.x OStlAvg.x OBlkAvg.x OPFAvg.x OWAvg.x OLAvg.x DayNumAvg.x DScoreAvg.x DFGMAvg.x DFGAAvg.x DFGM3Avg.x DFGA3Avg.x DFTMAvg.x DFTAAvg.x DORAvg.x DDRAvg.x DAstAvg.x DTOAvg.x DStlAvg.x DBlkAvg.x DPFAvg.x DWAvg.x DLAvg.x OScoreSum.y OFGMSum.y OFGASum.y OFGM3Sum.y OFGA3Sum.y OFTMSum.y OFTASum.y OORSum.y ODRSum.y OAstSum.y OTOSum.y OStlSum.y OBlkSum.y OPFSum.y OWSum.y OLSum.y DayNumSum.y DScoreSum.y DFGMSum.y DFGASum.y DFGM3Sum.y DFGA3Sum.y DFTMSum.y DFTASum.y DORSum.y DDRSum.y DAstSum.y DTOSum.y DStlSum.y DB

In [119]:
test.matchups.features %>%
    select(-Season,-DayNum,-TeamID.x,-TeamID.y) %>%
    write.csv('kerasfiles/test.matchups.ODstats.csv',row.names=F)

#### Delta Ratings

In [52]:
test.matchups %>% 
    inner_join(teams.stats.cumul,by=c('Season','DayNum','TeamID.x'='TeamID')) %>%
    mutate(pos=1) -> group1
test.matchups %>% 
    inner_join(teams.stats.cumul,by=c('Season','DayNum','TeamID.y'='TeamID')) %>%
    mutate(pos=2) -> group2
group1 %>% 
    bind_rows(group2) %>% 
    arrange(pos) %>% 
    group_by(Season,DayNum,TeamID.x,TeamID.y) %>% 
    mutate_at(vars(OScoreSum:DLAvg),list(~lag(.)-.)) %>% 
    ungroup %>% 
    filter(pos==2) %>%
    na.omit() %>%
    select(-pos) -> df.ratingdelta

In [53]:
df.ratingdelta %>% head

ID,Season,DayNum,TeamID.x,TeamID.y,Tourney,Target,OScoreSum,OFGMSum,OFGASum,⋯,DFTAAvg,DORAvg,DDRAvg,DAstAvg,DTOAvg,DStlAvg,DBlkAvg,DPFAvg,DWAvg,DLAvg
2014_1107_1301,2014,140,1107,1301,0,0,-294,-175,-312,⋯,-7.9503676,-2.79779412,-0.78860294,1.0790441,0.4393382,0.10845588,-0.498161765,-0.16176471,0.05514706,-0.05514706
2014_1107_1142,2014,140,1107,1142,0,0,253,66,20,⋯,-5.6229167,-0.29583333,-2.47291667,0.1770833,1.1354167,1.31041667,-0.568750000,3.61666667,-0.19583333,0.19583333
2014_1107_1397,2014,140,1107,1397,0,0,-143,-63,-130,⋯,-1.8437500,0.40625000,0.75000000,2.5312500,1.1250000,0.43750000,-0.812500000,-0.46875000,0.06250000,-0.06250000
2014_1107_1163,2014,140,1107,1163,0,0,-330,-111,-214,⋯,-4.2444853,-2.76838235,-0.61213235,0.8437500,-0.9430147,-0.06801471,-0.233455882,0.77941176,0.20220588,-0.20220588
2014_1107_1173,2014,140,1107,1173,0,0,-309,-129,-182,⋯,-6.7320076,0.48295455,-0.07291667,0.3740530,-0.6676136,1.14678030,-1.362689394,-0.08333333,0.13446970,-0.13446970
2014_1107_1196,2014,140,1107,1196,1,0,-292,-111,-159,⋯,-0.9209559,0.05514706,0.62316176,2.7261029,-2.0018382,0.19669118,0.001838235,-0.10294118,0.37867647,-0.37867647


In [54]:
df.ratingdelta %>% sample_n(5)

ID,Season,DayNum,TeamID.x,TeamID.y,Tourney,Target,OScoreSum,OFGMSum,OFGASum,⋯,DFTAAvg,DORAvg,DDRAvg,DAstAvg,DTOAvg,DStlAvg,DBlkAvg,DPFAvg,DWAvg,DLAvg
2018_1166_1254,2018,140,1166,1254,0,0,72,70,-17,⋯,-4.7302053,-2.1260997,0.9540567,1.07233627,-0.32649071,-0.4447703,0.2385142,-3.8152493,-0.1300098,0.1300098
2018_1120_1401,2018,140,1120,1401,0,0,268,-19,36,⋯,4.1875000,-1.1875000,0.5937500,-1.46875000,4.06250000,-0.1875000,-0.9375000,3.2187500,-0.1562500,0.1562500
2015_1172_1344,2015,140,1172,1344,0,0,58,36,-34,⋯,-0.3090909,0.8696970,2.2454545,0.55151515,-1.33030303,-0.5636364,0.8454545,-1.4212121,-0.1000000,0.1000000
2018_1301_1355,2018,140,1301,1355,0,0,98,94,158,⋯,3.0125000,2.2645833,-3.0041667,1.90625000,4.56666667,1.3187500,1.1979167,-1.4354167,0.1437500,-0.1437500
2017_1124_1277,2017,140,1124,1277,0,0,-147,-51,-105,⋯,-3.3313783,0.2737048,-3.8465298,0.09090909,-0.06451613,-0.5259042,0.5200391,-0.5826002,-0.1984360,0.1984360


In [59]:
group1 %>% filter(Season==2018,TeamID.x==1301,TeamID.y==1355)
group2 %>% filter(Season==2018,TeamID.y==1355,TeamID.x==1301)
df.ratingdelta %>% filter(Season==2018,TeamID.x==1301,TeamID.y==1355)

ID,Season,DayNum,TeamID.x,TeamID.y,Tourney,Target,OScoreSum,OFGMSum,OFGASum,⋯,DORAvg,DDRAvg,DAstAvg,DTOAvg,DStlAvg,DBlkAvg,DPFAvg,DWAvg,DLAvg,pos
2018_1301_1355,2018,140,1301,1355,0,0,2599,937,1993,⋯,11.03125,24.0625,13.90625,15,6.21875,4.03125,18.03125,0.34375,0.65625,1


ID,Season,DayNum,TeamID.x,TeamID.y,Tourney,Target,OScoreSum,OFGMSum,OFGASum,⋯,DORAvg,DDRAvg,DAstAvg,DTOAvg,DStlAvg,DBlkAvg,DPFAvg,DWAvg,DLAvg,pos
2018_1301_1355,2018,140,1301,1355,0,0,2501,843,1835,⋯,8.766667,27.06667,12,10.43333,4.9,2.833333,19.46667,0.2,0.8,2


ID,Season,DayNum,TeamID.x,TeamID.y,Tourney,Target,OScoreSum,OFGMSum,OFGASum,⋯,DFTAAvg,DORAvg,DDRAvg,DAstAvg,DTOAvg,DStlAvg,DBlkAvg,DPFAvg,DWAvg,DLAvg
2018_1301_1355,2018,140,1301,1355,0,0,98,94,158,⋯,3.0125,2.264583,-3.004167,1.90625,4.566667,1.31875,1.197917,-1.435417,0.14375,-0.14375


In [60]:
df.ratingdelta %>%
    select(-Season,-DayNum,-TeamID.x,-TeamID.y) %>%
    write.csv('kerasfiles/test.matchups.ODDeltaStats.csv',row.names=F)